In [1]:
!pip install alpaca-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 2.9 MB/s eta 0:00:00


In [23]:
from alpaca.data.historical import OptionHistoricalDataClient
from alpaca.data.requests import OptionBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.trading.requests import GetAssetsRequest
from alpaca.data.historical.stock import *
from alpaca.data.historical.crypto import *
from alpaca.data.historical.option import *
from alpaca.trading.client import *
import time
from datetime import datetime, timedelta, date
from zoneinfo import ZoneInfo
import pandas as pd

# Alpaca API credentials
api_key = "PK57L7BGUUMQZ7V25H5A"
secret_key = "uH5DCyc1WEwEIrQIRWVwKKdGAKjecHTJWfd9Z4mj"
base_url ='https://paper-api.alpaca.markets'

In [24]:
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=True, url_override=base_url)
trade_client.get_account()

{   'account_blocked': False,
    'account_number': 'PA3LA9EW27YE',
    'accrued_fees': '0',
    'buying_power': '201606.17',
    'cash': '100231.74',
    'created_at': datetime.datetime(2024, 5, 2, 14, 23, 14, 22192, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '101921.83',
    'id': UUID('5b0ec034-6506-42a6-b0bb-d48e51a0bf1d'),
    'initial_margin': '1118.75',
    'last_equity': '101916.0144',
    'last_maintenance_margin': '666.54',
    'long_market_value': '1963.79',
    'maintenance_margin': '671.25',
    'multiplier': '2',
    'non_marginable_buying_power': '99958.04',
    'options_approved_level': 3,
    'options_buying_power': '99958.04',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '101921.83',
    'regt_buying_power': '201606.17',
    'short

In [25]:
symbol = 'SPY'
# get list of assets which are options enabled
req = GetAssetsRequest(
  attributes = "options_enabled")
assets = trade_client.get_all_assets(req)
assets = pd.DataFrame.from_dict(list(map(dict,assets)))
assets[assets.symbol == symbol]

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement,attributes
1715,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,AssetClass.US_EQUITY,AssetExchange.ARCA,SPY,SPDR S&P 500 ETF Trust,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options, options_l..."


In [26]:
# setup stock historical data client
shd_client = StockHistoricalDataClient(api_key, secret_key, url_override = None)
# get historical bars by symbol

req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe= TimeFrame(amount = 1, unit = TimeFrameUnit.Day), # specify timeframe
    start = '2024-01-01',                                         # specify start datetime, default=the beginning of the current day.
    #limit = 1000                                               # specify limit
)


ohlc_data = shd_client.get_stock_bars(req).df
ohlc_data = ohlc_data.loc[symbol]
ohlc_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2024-01-02 05:00:00+00:00,472.16,473.67,470.4900,472.65,123007793.0,657585.0,472.044088
2024-01-03 05:00:00+00:00,470.43,471.19,468.1700,468.79,103595966.0,656572.0,469.620528
2024-01-04 05:00:00+00:00,468.30,470.96,467.0500,467.28,84232169.0,536471.0,468.783715
2024-01-05 05:00:00+00:00,467.49,470.44,466.4300,467.92,86118913.0,562579.0,468.323224
2024-01-08 05:00:00+00:00,468.43,474.75,468.3000,474.60,74879074.0,523806.0,472.074423
...,...,...,...,...,...,...,...
2025-05-15 04:00:00+00:00,585.56,590.97,585.0969,590.46,71268145.0,737359.0,588.585752
2025-05-16 04:00:00+00:00,591.25,594.50,589.2800,594.20,76052101.0,645314.0,592.388584
2025-05-19 04:00:00+00:00,588.10,595.54,588.1000,594.85,68168509.0,662361.0,593.168182


In [28]:
import numpy as np

In [29]:
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe= TimeFrame(amount = 1, unit = TimeFrameUnit.Day), # specify timeframe
    start = '2023-01-01',                                         # specify start datetime, default=the beginning of the current day.
    #limit = 1000                                               # specify limit
    end_date = '2023-12-31'
)


past_data = shd_client.get_stock_bars(req).df
past_data = past_data.loc[symbol]
past_data['log_returns'] = np.log(past_data.close).diff()
hv = past_data['log_returns'].std()*np.sqrt(252)*100

In [30]:
from alpaca.trading.enums import *
from datetime import datetime
from zoneinfo import ZoneInfo
import pandas as pd

In [32]:
from alpaca.trading.enums import *

In [56]:
symbol = 'SPY'
underlying_symbol = [symbol]
from alpaca.trading.enums import *
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now - timedelta(days = 30)
day120 = now + timedelta(days = 120)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbol,                     # specify underlying symbols
    expiration_date =None,                                      # specify expiration date (specified date + 1 day range)
    #expiration_date_gte = "2025-01-01",                           # we can pass date object
    #expiration_date_lte = day120.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = None,                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 1000,                                                 # specify limit
    page_token = None,                                           # specify page
    open_interest = 1
    )
res = trade_client.get_option_contracts(req)
options_df = pd.DataFrame(list(map(dict,res.option_contracts)))
#options_df = options_df[options_df.open_interest.astype(float) >= 10]
options_df

,id,symbol,name,status,tradable,expiration_date,root_symbol,underlying_symbol,underlying_asset_id,type,style,strike_price,size,open_interest,open_interest_date,close_price,close_price_date
0,01a06261-52d6-49d9-a7aa-99215e19d7af,SPY250521C00350000,SPY May 21 2025 350 Call,AssetStatus.ACTIVE,True,2025-05-21,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,350.0,100,1,2025-05-19,243.37,2025-05-20
1,e865d32f-0e18-4163-b49f-a020de720493,SPY250521C00355000,SPY May 21 2025 355 Call,AssetStatus.ACTIVE,True,2025-05-21,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,355.0,100,None,None,None,None
2,81c1994e-84c5-4353-9763-aa233a39fda2,SPY250521C00360000,SPY May 21 2025 360 Call,AssetStatus.ACTIVE,True,2025-05-21,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,360.0,100,None,None,None,None
3,da5282e9-1c52-4135-9b17-e56d1fa189e3,SPY250521C00365000,SPY May 21 2025 365 Call,AssetStatus.ACTIVE,True,2025-05-21,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,365.0,100,None,None,None,None
4,1781ff79-9606-4479-b24e-6ad01af05121,SPY250521C00370000,SPY May 21 2025 370 Call,AssetStatus.ACTIVE,True,2025-05-21,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,370.0,100,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,5c1e0013-8dc1-4896-a69d-8777edc7177e,SPY250523P00680000,SPY May 23 2025 680 Put,AssetStatus.ACTIVE,True,2025-05-23,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.PUT,ExerciseStyle.AMERICAN,680.0,100,None,None,None,None
956,b5cedc6c-503b-4805-8e9a-638a87ee69c7,SPY250523P00685000,SPY May 23 2025 685 Put,AssetStatus.ACTIVE,True,2025-05-23,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.PUT,ExerciseStyle.AMERICAN,685.0,100,None,None,None,None
957,eb58a37c-5934-4e98-abcd-8ec6627c357a,SPY250523P00690000,SPY May 23 2025 690 Put,AssetStatus.ACTIVE,True,2025-05-23,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.PUT,ExerciseStyle.AMERICAN,690.0,100,None,None,None,None
958,7fb84a94-8d8d-48c7-a6d3-3c4efb3ac8b0,SPY250523P00695000,SPY May 23 2025 695 Put,AssetStatus.ACTIVE,True,2025-05-23,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.PUT,ExerciseStyle.AMERICAN,695.0,100,None,None,None,None


In [59]:
option_symbols = []

for strike in range(455, 510, 5):  # 510 because range is exclusive on the end
    strike_formatted = f"{int(strike * 1000):08d}"  # 8-digit, 3 implied decimals
    option_symbol = f"SPY240215C{strike_formatted}"
    option_symbols.append(option_symbol)

print(option_symbols)

['SPY240215C00455000', 'SPY240215C00460000', 'SPY240215C00465000', 'SPY240215C00470000', 'SPY240215C00475000', 'SPY240215C00480000', 'SPY240215C00485000', 'SPY240215C00490000', 'SPY240215C00495000', 'SPY240215C00500000', 'SPY240215C00505000']


In [61]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

symbol = "SPY"

# Alpaca API credentials
api_key = "PK57L7BGUUMQZ7V25H5A"
secret_key = "uH5DCyc1WEwEIrQIRWVwKKdGAKjecHTJWfd9Z4mj"

# Initialize Alpaca data client
shd_client = StockHistoricalDataClient(api_key, secret_key)

# Get SPY historical data from Feb 1 to Apr 1
req = StockBarsRequest(
    symbol_or_symbols=[symbol],
    timeframe=TimeFrame(1, TimeFrameUnit.Day),
    start='2024-05-01',
    end='2025-05-01'
)
bars = shd_client.get_stock_bars(req).df
bars = bars.loc[symbol]

# Prepare final output list
all_option_symbols = []

# Loop over each trading day
for date, row in bars.iterrows():
    close_price = row['close']

    # Round to nearest 5
    base_strike = round(close_price / 5) * 5

    # Generate list of strike prices from -30 to +30
    strikes = [base_strike + x for x in range(-30, 35, 5)]

    # Format expiration date as YYMMDD
    expiration_str = date.strftime('%y%m%d')

    # Generate option symbols
    symbols_for_day = [
        f"SPY{expiration_str}C{int(strike * 1000):08d}" for strike in strikes
    ]

    all_option_symbols.append({
        "date": date.strftime('%Y-%m-%d'),
        "symbols": symbols_for_day
    })

# Output: list of dicts with 'date' and list of option symbols
# You can save or print this
print(all_option_symbols)

[{'date': '2024-05-01', 'symbols': ['SPY240501C00470000', 'SPY240501C00475000', 'SPY240501C00480000', 'SPY240501C00485000', 'SPY240501C00490000', 'SPY240501C00495000', 'SPY240501C00500000', 'SPY240501C00505000', 'SPY240501C00510000', 'SPY240501C00515000', 'SPY240501C00520000', 'SPY240501C00525000', 'SPY240501C00530000']}, {'date': '2024-05-02', 'symbols': ['SPY240502C00475000', 'SPY240502C00480000', 'SPY240502C00485000', 'SPY240502C00490000', 'SPY240502C00495000', 'SPY240502C00500000', 'SPY240502C00505000', 'SPY240502C00510000', 'SPY240502C00515000', 'SPY240502C00520000', 'SPY240502C00525000', 'SPY240502C00530000', 'SPY240502C00535000']}, {'date': '2024-05-03', 'symbols': ['SPY240503C00480000', 'SPY240503C00485000', 'SPY240503C00490000', 'SPY240503C00495000', 'SPY240503C00500000', 'SPY240503C00505000', 'SPY240503C00510000', 'SPY240503C00515000', 'SPY240503C00520000', 'SPY240503C00525000', 'SPY240503C00530000', 'SPY240503C00535000', 'SPY240503C00540000']}, {'date': '2024-05-06', 'symbol

In [65]:
from alpaca.data.historical import OptionHistoricalDataClient
from alpaca.data.requests import OptionBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
from datetime import datetime, timedelta

client = OptionHistoricalDataClient(api_key, secret_key)

# Only take the first 10 entries
limited_symbols = all_option_symbols[:10]

# Store all results here
all_data = []

for day_entry in limited_symbols:
    # expiration date (same as option expiration, assumed to be that day's date)
    expiration_date = datetime.strptime(day_entry['date'], "%Y-%m-%d")
    start_date = expiration_date - timedelta(days=7)  # 5 trading days ≈ 7 calendar days

    for option_symbol in day_entry['symbols']:
        try:
            request = OptionBarsRequest(
                symbol_or_symbols=option_symbol,
                timeframe=TimeFrame.Day,
                start=start_date,
                end=expiration_date
            )
            bars = client.get_option_bars(request)
            df = bars.df.reset_index()

            if not df.empty:
                df['option_symbol'] = option_symbol
                df['expiration_date'] = expiration_date.strftime('%Y-%m-%d')
                all_data.append(df)

        except Exception as e:
            print(f"Error fetching {option_symbol}: {e}")

# Combine all into a single DataFrame
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.to_csv("spy_options_subset.csv", index=False)
    print("Saved as 'spy_options_subset.csv'")
    print(final_df.head())
else:
    print("No data collected.")


Saved as 'spy_options_subset.csv'
               symbol                 timestamp   open   high    low  close  \
0  SPY240501C00470000 2024-04-24 04:00:00+00:00  34.53  34.53  34.53  34.53   
1  SPY240501C00470000 2024-04-29 04:00:00+00:00  39.85  39.85  39.85  39.85   
2  SPY240501C00470000 2024-04-30 04:00:00+00:00  38.47  38.47  32.16  32.16   
3  SPY240501C00480000 2024-04-24 04:00:00+00:00  26.60  26.60  26.60  26.60   
4  SPY240501C00480000 2024-04-25 04:00:00+00:00  19.45  25.00  19.42  25.00   

   volume  trade_count       vwap       option_symbol expiration_date  
0     2.0          1.0  34.530000  SPY240501C00470000      2024-05-01  
1     1.0          1.0  39.850000  SPY240501C00470000      2024-05-01  
2     9.0          6.0  34.015556  SPY240501C00470000      2024-05-01  
3     1.0          1.0  26.600000  SPY240501C00480000      2024-05-01  
4    87.0          4.0  22.670690  SPY240501C00480000      2024-05-01  


In [66]:
final_df.head(20)

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,option_symbol,expiration_date
0,SPY240501C00470000,2024-04-24 04:00:00+00:00,34.53,34.53,34.53,34.53,2.0,1.0,34.530000,SPY240501C00470000,2024-05-01
1,SPY240501C00470000,2024-04-29 04:00:00+00:00,39.85,39.85,39.85,39.85,1.0,1.0,39.850000,SPY240501C00470000,2024-05-01
2,SPY240501C00470000,2024-04-30 04:00:00+00:00,38.47,38.47,32.16,32.16,9.0,6.0,34.015556,SPY240501C00470000,2024-05-01
3,SPY240501C00480000,2024-04-24 04:00:00+00:00,26.60,26.60,26.60,26.60,1.0,1.0,26.600000,SPY240501C00480000,2024-05-01
4,SPY240501C00480000,2024-04-25 04:00:00+00:00,19.45,25.00,19.42,25.00,87.0,4.0,22.670690,SPY240501C00480000,2024-05-01
5,SPY240501C00480000,2024-04-26 04:00:00+00:00,28.52,29.88,28.49,29.88,32.0,8.0,28.556563,SPY240501C00480000,2024-05-01
6,SPY240501C00480000,2024-04-29 04:00:00+00:00,29.87,29.87,29.87,29.87,1.0,1.0,29.870000,SPY240501C00480000,2024-05-01
7,SPY240501C00480000,2024-04-30 04:00:00+00:00,26.60,26.60,22.10,22.10,8.0,8.0,24.768750,SPY240501C00480000,2024-05-01
8,SPY240501C00485000,2024-04-30 04:00:00+00:00,23.30,24.15,23.30,24.15,200.0,7.0,23.712500,SPY240501C00485000,2024-05-01
9,SPY240501C00490000,2024-04-24 04:00:00+00:00,17.42,18.21,15.07,15.91,33.0,9.0,16.727273,SPY240501C00490000,2024-05-01


In [58]:
client = OptionHistoricalDataClient(api_key, secret_key)

# Define your time range
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 5, 1)

# Full OCC option symbol for SPY call: Jan 17, 2025 $450 call
option_symbol = 'SPY240215C00475000'

request = OptionBarsRequest(
    symbol_or_symbols=option_symbol,
    timeframe=TimeFrame.Day,
    start=start_date,
    end=end_date
)

bars = client.get_option_bars(request)
df = bars.df.reset_index()

df.to_csv("spy_option_eod.csv", index=False)
print("Datei gespeichert als 'spy_option_eod.csv'")

print(df.head())

Datei gespeichert als 'spy_option_eod.csv'
               symbol                 timestamp   open   high    low  close  \
0  SPY240215C00475000 2024-02-09 05:00:00+00:00  24.62  24.62  24.62  24.62   
1  SPY240215C00475000 2024-02-12 05:00:00+00:00  27.42  27.42  26.01  26.01   
2  SPY240215C00475000 2024-02-13 05:00:00+00:00  19.32  20.80  18.13  18.13   

   volume  trade_count       vwap  
0     1.0          1.0  24.620000  
1     7.0          2.0  26.412857  
2    65.0         20.0  20.285692  


In [ ]:
from google.colab import files
files.download('spy_option_eod.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>